# Brandenburg

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Brandenburg is `DE4`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [ ]:
import pandas as pd
from pandas.errors import ParserError
import os
from pprint import pprint
from tqdm import tqdm
from typing import Union, Dict, Tuple
import zipfile
from datetime import datetime as dt
from io import StringIO
import warnings
from dateparser import parse

from camelsp import Bundesland

The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [ ]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('Brandenburg').input_path
BASE

### Metadata reader

Define the function that extracts / reads and eventually merges all metadata for this federal state. You can develop the function here, without using the Bundesland context and then later use the context to pass extracted metadata. The Context has a function for saving *raw* metadata, that takes a `pandas.DataFrame` and needs you to identify the id column.
Here, *raw* refers to provider metadata, that has not yet been transformed into the CAMELS-de Metadata schema.

Nice, Brandenburg is quite cool. This will be hard to parse. Let's first extract the big Zip, because it's a zip of Excel, with MANY sheets. To make thing more complicated, they splitted the Excel files into two files, I guess because they got too large (haha).

In [ ]:
# extract the ZIP in place
if not os.path.exists(os.path.join(BASE, 'Q.TagWerte_1.xlsx')):
    with zipfile.ZipFile(os.path.join(BASE, 'Anlage_4_W_Q-TagWerte.zip')) as z:
        for f in z.filelist:
            z.extract(f, BASE)

Extract the metadata from each sheet, for this, first read in all files and create a mapping from station ids to the sheet in the respective excel_file. 

Ah, nice, why call them by their name if you can just call them 'Zeitreihe_1', 'Zeitreihe_2' and so on?

In [ ]:
q1 = pd.ExcelFile(os.path.join(BASE, 'Q.TagWerte_1.xlsx'))
q2 = pd.ExcelFile(os.path.join(BASE, 'Q.TagWerte_2.xlsx'))
w1 = pd.ExcelFile(os.path.join(BASE, 'W.TagWerte_1.xlsx'))
w2 = pd.ExcelFile(os.path.join(BASE, 'W.TagWerte_2.xlsx'))

In [ ]:
name = 'Zeitreihe_1'
df = q1.parse(name, header=None, usecols=[0,1,2,3])
df

This will be a tough sunday.

The header is changing its size. I hope the blocks stay the same...

In [ ]:
def parse_dirty_dataframe(df: pd.DataFrame, variable: str, skip_data: bool = False) -> Tuple[Dict, pd.DataFrame]:
    """
    Parse the dirty dataframe directly read from the excel sheets.

    """
    # get all empty row, as they separate the block, of course
    null_idx = df[df.isnull().all(axis=1) == True].index.to_list()

    # mark all block as None for now
    ID = None
    base = {}
    loc = {}
    co = {}
    dat = None

    # We expect three blocks here, base metadata, location metadata, coordinates and data
    for i, (lo, up) in enumerate(zip([0] + null_idx, null_idx + [len(df)])):
        # extract the block
        block = df.iloc[lo:up, :]
        
        # switch the block
        # data block - skipping for now
        if i == 3 and not skip_data:
            # on i==0, the ID should be filled, otherwise we have to handcraft this data
            if ID is None:
                continue
            
            # get the block
            block = block.dropna(axis=0, how='all').dropna(axis=1, how='all')

            # set the header
            block.columns = block.iloc[0, :]
            block.drop(block.index[0], axis=0, inplace=True)

            # transform the data as everything is string, of course
            dat = pd.DataFrame({
                'date': [d.date() if isinstance(d, dt) else parse(d) for d in block.iloc[:, 0].values],
                variable.lower(): block.iloc[:, 1].astype(float),
                'flag': [fl.strip().lower() == 'geprüft' for fl in block.iloc[:, 3].values]
            })
        
        # base data or Standord
        if i == 0 or i == 1:
            block = block.iloc[:, :2].dropna(axis=1, how='all').dropna(axis=0, how='all')
            block = block.set_index(0).T.to_dict(orient='records')
            if len(block) > 0:
                if i == 0:
                    if 'ID' in block[0].keys():
                        ID = str(block[0]['ID'])
                    else:
                        warnings.warn(f"Block #{i + 1}: No ID found. This will skip the data for this station.")
                    base = block[0]
                else:
                    loc = block[0]
            else:
                warnings.warn(f"Block #{i + 1}: did not yield the correct shape. Please check the file. Skipping.")
        
        # Koordinaten
        elif i == 2:
            # drop NaNs on both axis
            block = block.dropna(axis=1, how='all').dropna(axis=0, how='all')
            
            # rename the CRS header
            block.iloc[0, 0] = 'CRS'
            block.columns = block.iloc[0]
            block.drop(block.index[0], axis=0, inplace=True)
            block = block.to_dict(orient='records')

            if len(block) > 0:
                co = block[0]
            else:
                warnings.warn(f"Block #{i + 1} did not yield the correct shape. Please check the file. Skipping.")

    # now merge the metadata
    meta = {**base, **loc, **co}

    # if there was no metadata, set meta None again
    if len(meta.keys()) == 0:
        meta = None
    
    # finally return all we got
    return meta, dat


# Test the stuff
parse_dirty_dataframe(df, 'q', skip_data=False)


## Generate metadata and data in Memory

I hope all the stuff can be put into Memory at once, otherwise we have to chunk it.

In [ ]:
# empty container for the data
metadata, data, warns = [], [], []

with warnings.catch_warnings(record=True) as wa:
    # extract from each of the four excel sheets
    for variable, xls in zip(('q', 'q', 'w', 'w'), (q1, q2, w1, w2)):
        # go for each sheet
        for sheet_name in tqdm(xls.sheet_names):
            # load the dirty sheet
            df = xls.parse(sheet_name, header=None, usecols=[0,1,2,3])

            # parse it
            meta, dat = parse_dirty_dataframe(df, variable)

            if meta is not None:
                metadata.append(meta),
                data.append(dat)

    # copy warnings
    warns.extend(wa)

print(f"metadata length: {len(metadata)}    data files: {len(data)}      warnings:{len(warns)}")

## There is more

We have not only Anlage 4, but also Anlage 3. Check this file out:

In [ ]:
# define the function 
def read_meta(base_path) -> pd.DataFrame:
    path = os.path.join(base_path, 'Anlage_3.xlsx')
    meta = pd.read_excel(path)
    return meta

# test it here
other_meta = read_meta(BASE)
pmeta = pd.DataFrame(metadata)

# merge with the other, more interesting metadata
#meta = pd.merge(pd.DataFrame(metadata), other_meta, left_on='ID', right_on='station_no', how='inner')

print(f"Overlapping names: {any([n in pmeta.Name.values for n in other_meta.station_name])}")
print(f"Overlapping IDs: {any([str(i) in pmeta.Messstellennummer.values.tolist() for i in other_meta.station_no])}")

In [ ]:
# join both metadata together
all_meta = pmeta.join(other_meta.set_index('station_no'), on='Messstellennummer', how='left')
all_meta

In [ ]:
# the id column will be Messstellennummer
id_column = 'Messstellennummer'

Check that we have an ID everywhere

In [ ]:
print(f"All stations have an id: {all(['Messstellennummer' in m for m in metadata])}")

### Finally run

Save the stuff using the context

In [ ]:
with Bundesland('Brandenburg') as bl:
    # save the metadata
    bl.save_raw_metadata(all_meta, id_column, overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())
    
    # go for each of the files
    for station_meta, station_data in tqdm(zip(metadata, data), total=len(metadata)):
        # get the provider id
        provider_id = station_meta['Messstellennummer']
        
        # save
        bl.save_timeseries(station_data, provider_id)

    # check if there were warnings (there are warnings)
    if len(warns) > 0:
        log_path = bl.save_warnings(warns)
        print(f"There were warnings during the processing. The log can be found at: {log_path}")
